In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers.string import StrOutputParser
from langchain_openai import ChatOpenAI
import json
from helper_funcs import get_api_key_url, read_txt_file_to_string, json_to_table

# --- Setup ---
API_KEY, API_URL = get_api_key_url(client="OPENAI")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2, api_key=API_KEY)

# --- Documenten ---
doc1 = read_txt_file_to_string("documents/doc1.txt")
doc2 = read_txt_file_to_string("documents/doc2.txt")

# building prompt
criteria = (
    "Governance structuur\n"
    "Risicomanagement\n"
    "Incident response\n"
    "Outsourcing beleid"
)

table_description = {
    "Criterium": "Zoals gegeven",
    "Overeenstemming": "Mate van overeenstemming tussen document A en B, bijvoorbeeld 'Volledig in lijn', 'Gedeeltelijk', of 'Niet in lijn'.",
    "Analyse Document A": "Korte samenvatting van hoe document A het criterium behandelt.",
    "Analyse Document B": "Korte samenvatting van hoe document B het criterium behandelt.",
    "Toelichting": "Een korte uitleg van de overeenkomsten of verschillen tussen de documenten.",
    "Referentie A": "Een citaat of tekstfragment (met paginanummer indien mogelijk) uit document A.",
    "Referentie B": "Een citaat of tekstfragment (met paginanummer indien mogelijk) uit document B."
}

prompt_template = ChatPromptTemplate.from_template(f"""
Je bent een compliance-expert gespecialiseerd in DORA, ISO27001 en vergelijkbare standaarden.
Je vergelijkt twee documenten aan de hand van specifieke criteria.
Vul onderstaande tabelstructuur in met de juiste informatie op basis van de inhoud van de documenten.

Document A:
\"\"\"{{doc1}}\"\"\"

Document B:
\"\"\"{{doc2}}\"\"\"

Criteria:
{{criteria}}

Antwoord in geldig JSON-formaat als volgt:
[
    {{table_description}},
    ...
]
""")

# Chain prompt
chain = prompt_template | llm | StrOutputParser()
output = chain.invoke({
    "doc1": doc1,
    "doc2": doc2,
    "criteria": json.dumps(criteria, ensure_ascii=False),
    "table_description": table_description
})


df = json_to_table(output, print_as_md=True, save_as_csv=True)



📊 Vergelijkingsoverzicht:

| Criterium            | Overeenstemming   | Analyse Document A                                                                                                                                                            | Analyse Document B                                                                                                                                                               | Toelichting                                                                                                                                                  | Referentie A                                                                                                                    | Referentie B                                                                                              |
|:---------------------|:------------------|:----------------------------------------------------------------------------------------------------------------------------------

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers.string import StrOutputParser
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import json

from helper_funcs import get_api_key_url, read_txt_file_to_string, json_to_table

# --- API setup ---
API_KEY, API_URL = get_api_key_url(client="OPENAI")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2, api_key=API_KEY)
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# --- Documenten inladen ---
doc1 = read_txt_file_to_string("documents/doc1.txt")
doc2 = read_txt_file_to_string("documents/doc2.txt")

# --- Rubricvragen ---
criteria = (
    "1. Vorm: Is de tekst helder, zakelijk en passend bij de doelgroep?\n"
    "2. Structuur: Heeft de tekst een logische opbouw met duidelijke alineas en goede overgangen?\n"
    "3. Inhoud: Zijn de hoofdgedachte en argumenten overtuigend en relevant?\n"
    "4. Stijl: Is de toon professioneel en consequent?\n"
    "5. Taalgebruik: Zijn spelling, grammatica en zinsbouw correct?"
)

# --- Promptdefinitie ---
prompt_template = ChatPromptTemplate.from_template("""
Je bent een ervaren docent Nederlands gespecialiseerd in zakelijke communicatie.
Je beoordeelt twee zakelijke teksten aan de hand van een vaste rubric met vragen over vorm, inhoud en structuur.

Beoordeel elk criterium afzonderlijk voor beide teksten.
Voor elke vraag uit de rubric geef je:
- een beoordeling per tekst (bijv. "goed", "voldoende", "onvoldoende")
- een korte toelichting
- citeer een relevant tekstfragment waarop de beoordeling gebaseerd is

Document A:
\"\"\"{doc1}\"\"\"

Document B:
\"\"\"{doc2}\"\"\"

Rubricvragen:
{criteria}

Antwoord in geldig JSON-formaat als volgt:
[
    {{
        "criterium": "...",
        "beoordeling_document_A": "...",
        "toelichting_A": "...",
        "referentie_A": "...",
        "beoordeling_document_B": "...",
        "toelichting_B": "...",
        "referentie_B": "..."
    }},
    ...
]
""")

# --- Chain bouwen ---
chain = prompt_template | llm | StrOutputParser()

# --- LLM aanroepen ---
output = chain.invoke({
    "doc1": doc1,
    "doc2": doc2,
    "criteria": json.dumps(criteria, ensure_ascii=False)
})

# --- Uitvoeren ---
df = json_to_table(output, print_as_md=True, save_as_csv=False)



📊 Vergelijkingsoverzicht:

| criterium                                                                                  | beoordeling_document_A   | toelichting_A                                                                                                           | referentie_A                                                                                                                                   | beoordeling_document_B   | toelichting_B                                                                                                          | referentie_B                                                                                                                            |
|:-------------------------------------------------------------------------------------------|:-------------------------|:------------------------------------------------------------------------------------------------------------------------|:-----------------------------------------------------